<a href="https://colab.research.google.com/github/FindPrint/Demo/blob/main/demo_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Demo: Stochastic equation with memory and dynamic dimension
# Minimal Colab prototype

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq

print("✅ Libraries loaded. Upload your CSV in the next cell or use the example dataset.")

In [ ]:
import pandas as pd
import numpy as np

# Exemple intégré si aucun fichier n'est uploadé
example = pd.DataFrame({
    "time": np.linspace(0, 10, 500),
    "signal": np.sin(2*np.pi*0.5*np.linspace(0, 10, 500)) + 0.1*np.random.randn(500)
})
example.to_csv("example.csv", index=False)

print("👉 You can upload your own CSV (at least 2 columns), or use the example.csv provided.")

# Upload via Colab
try:
    from google.colab import files
    uploaded = files.upload()
except ImportError:
    uploaded = {}

# Si un fichier est uploadé, on le lit, sinon on prend l'exemple
if uploaded:
    filename = list(uploaded.keys())[0]
    df = pd.read_csv(filename)
else:
    df = pd.read_csv("example.csv")

# ✅ Normalisation automatique
if df.shape[1] < 2:
    raise ValueError("❌ Your CSV must contain at least 2 columns.")

# On garde seulement les 2 premières colonnes
df = df.iloc[:, :2]

# On renomme en 't' et 'phi'
df.columns = ['t', 'phi']

# On supprime les valeurs manquantes
df = df.dropna().reset_index(drop=True)

# Vérification des types numériques
if not np.issubdtype(df['t'].dtype, np.number) or not np.issubdtype(df['phi'].dtype, np.number):
    raise TypeError("⚠️ Error: Columns 't' and 'phi' must contain numeric values only.")

# Extraction des vecteurs
t = df['t'].values
phi = df['phi'].values

print("✅ Data loaded and cleaned:", df.shape)
print(df.head())

In [ ]:
# Amplitude observée = moyenne absolue sur les 20% finaux
phi_final = phi[int(0.8 * len(phi)):]
amp_obs = np.mean(np.abs(phi_final))
print("Observed amplitude:", amp_obs)

In [ ]:
# Estimation simple de alpha_mean via fréquence dominante
yf = rfft(phi - np.mean(phi))
xf = rfftfreq(len(phi), d=(t[1] - t[0]))
f_dom = xf[np.argmax(np.abs(yf))]
alpha_mean = f_dom * np.var(phi)

# Facteur d’échelle ajustable
factor = 100.0
alpha_scaled = factor * alpha_mean
amp_theo = np.sqrt(alpha_scaled) if alpha_scaled > 0 else 0

print("α_mean (spectral):", alpha_mean)
print("Theoretical amplitude:", amp_theo)

In [ ]:
# Erreur relative
error = abs(amp_obs - amp_theo) / amp_theo if amp_theo > 0 else None

# Graphe
plt.figure(figsize=(10,5))
plt.plot(t, phi, label="Real signal")
plt.axhline(amp_obs, color="green", linestyle="--", label=f"Observed amplitude = {amp_obs:.2f}")
plt.axhline(amp_theo, color="red", linestyle="--", label=f"Theoretical amplitude = {amp_theo:.2f}")
plt.xlabel("Time")
plt.ylabel("φ(t)")
plt.title("Observed vs Theoretical Amplitude")
plt.legend()
plt.show()

print("Relative error:", f"{error:.2%}" if error is not None else "N/A")